## Demo 2.1: Exposing vcluster using Loadbalancer service

In [1]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   6m23s
kube-node-lease      Active   6m25s
kube-public          Active   6m25s
kube-system          Active   6m25s
local-path-storage   Active   6m19s
metallb-system       Active   5m28s


### Create vcluster and expose it as a Loadbalancer service

In [2]:
! vcluster create vc1 -n host-namespace --expose

[info]   Creating namespace host-namespace
[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/3406161568 --namespace host-namespace --install --repository-config='' --values /tmp/2715273990
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


In [11]:
! kubectl get all -n host-namespace

NAME                                               READY   STATUS    RESTARTS   AGE
pod/coredns-5df468b6b7-slfkn-x-kube-system-x-vc1   1/1     Running   0          6s
pod/vc1-0                                          2/2     Running   0          72s

NAME                                        TYPE           CLUSTER-IP      EXTERNAL-IP      PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1        ClusterIP      10.96.179.102   <none>           53/UDP,53/TCP,9153/TCP   6s
service/vc1                                 LoadBalancer   10.96.216.159   172.18.255.201   443:32226/TCP            72s
service/vc1-headless                        ClusterIP      None            <none>           443/TCP                  72s
service/vc1-node-kcd-vcluster-demo-worker   ClusterIP      10.96.209.17    <none>           10250/TCP                6s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     72s


##### Accessing Vcluster using Cluster Admin Kubeconfig

In [12]:
! vcluster connect vc1 --namespace host-namespace

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: ./kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig ./kubeconfig.yaml get namespaces`


In [13]:
! kubectl get ns --kubeconfig ./kubeconfig.yaml

NAME              STATUS   AGE
default           Active   96s
kube-system       Active   96s
kube-public       Active   96s
kube-node-lease   Active   96s


##### Create a workload inside the vcluster

In [14]:
! kubectl create ns ns1 --kubeconfig ./kubeconfig.yaml
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./kubeconfig.yaml

namespace/ns1 created
deployment.apps/sample created


In [22]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

NAME                      READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-t9phf   1/1     Running   0          43s
sample-7f75ccc884-lfshf   1/1     Running   0          43s


##### Accessing vcluster using a Read Only Kubeconfig

In [23]:
 ! vcluster connect vc1 --namespace host-namespace \
    --service-account kube-system/my-user --cluster-role view --kube-config svc_account_kubeconfig.yaml

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Create service account token for kube-system/my-user
Forwarding from 127.0.0.1:11000 -> 8443
Forwarding from [::1]:11000 -> 8443
Handling connection for 11000
[done] √ Created service account kube-system/my-user
[done] √ Created cluster role binding for cluster role view
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: svc_account_kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig svc_account_kubeconfig.yaml get namespaces`


In [24]:
! kubectl get deployments -n ns1 --kubeconfig ./svc_account_kubeconfig.yaml

NAME     READY   UP-TO-DATE   AVAILABLE   AGE
sample   2/2     2            2           93s


In [25]:
! kubectl delete deployment sample -n ns1 --kubeconfig ./svc_account_kubeconfig.yaml

Error from server (Forbidden): deployments.apps "sample" is forbidden: User "system:serviceaccount:kube-system:my-user" cannot delete resource "deployments" in API group "apps" in the namespace "ns1"


##### Accessing vcluster without using vcluster CLI

In [26]:
%%bash
 kubectl get secret vc-vc1 -n host-namespace -o go-template --template='{{.data.config}}' | base64 -d

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUJlRENDQVIyZ0F3SUJBZ0lCQURBS0JnZ3Foa2pPUFFRREFqQWpNU0V3SHdZRFZRUUREQmhyTTNNdGMyVnkKZG1WeUxXTmhRREUyTlRNMk5EVXdOVFV3SGhjTk1qSXdOVEkzTURrMU1EVTFXaGNOTXpJd05USTBNRGsxTURVMQpXakFqTVNFd0h3WURWUVFEREJock0zTXRjMlZ5ZG1WeUxXTmhRREUyTlRNMk5EVXdOVFV3V1RBVEJnY3Foa2pPClBRSUJCZ2dxaGtqT1BRTUJCd05DQUFSVEcxYnJLRmZIclk0T2tkN3FMSGJONVU4eHBxeFJ2MHEreXp6M3cyT2IKK0UrZzNTaGxscU5tUTdLdWE1dnZ3bGkzbytwRFh5YzNNYlJjVUdpVms4NFZvMEl3UURBT0JnTlZIUThCQWY4RQpCQU1DQXFRd0R3WURWUjBUQVFIL0JBVXdBd0VCL3pBZEJnTlZIUTRFRmdRVXpHREU3RStmRTcyeXdLTFVEbkNHCkRtR3FMcDB3Q2dZSUtvWkl6ajBFQXdJRFNRQXdSZ0loQVBsUjVCV0lGZ0x5NUlDM0xhMnFqbXZibFFXNXp3aEQKTWIwbm1YVWEwR2h0QWlFQXVqRWFWc0ROamgvNitLZnNpL0VUckxEdlllU0dxaTlkMGxBN2RzNXhCQVk9Ci0tLS0tRU5EIENFUlRJRklDQVRFLS0tLS0K
    server: https://localhost:8443
  name: local
contexts:
- context:
    cluster: local
    namespace: default
    user: user
  name: Default
current-context: Default
kin

### Cleanup

In [27]:
! vcluster delete vc1 -n host-namespace
! kubectl delete ns host-namespace

[info]   Delete helm chart with helm delete vc1 --namespace host-namespace --kubeconfig /tmp/3457935168 --repository-config=''
[done] √ Successfully deleted virtual cluster vc1 in namespace host-namespace
[done] √ Successfully deleted virtual cluster pvc data-vc1-0 in namespace host-namespace
namespace "host-namespace" deleted


In [28]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   13m
kube-node-lease      Active   13m
kube-public          Active   13m
kube-system          Active   13m
local-path-storage   Active   13m
metallb-system       Active   12m
